In [1]:
# install spark
!pip install pyspark

You should consider upgrading via the '/Users/fengzheng/Desktop/Coding/Practice/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
# create spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/28 18:07:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Load the flight dataset
flight_df = spark.read.option('header','true').csv('data_raw/flight_raw')

# Create timestamp column
from pyspark.sql.functions import *
flight_df = flight_df.withColumn('Date',to_date(concat_ws('-', flight_df.Year, flight_df.Month,flight_df.DayofMonth)))

# Select needed columns
flight_df = flight_df.select('Date','Airline','Origin','Dest','Cancelled','OriginStateName','DestStateName')\
                        .sort('Date')

# Print schema
flight_df.printSchema()
flight_df.show(5)

root
 |-- Date: date (nullable = true)
 |-- Airline: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- OriginStateName: string (nullable = true)
 |-- DestStateName: string (nullable = true)



+----------+--------------------+------+----+---------+---------------+-------------+
|      Date|             Airline|Origin|Dest|Cancelled|OriginStateName|DestStateName|
+----------+--------------------+------+----+---------+---------------+-------------+
|2019-01-01|ExpressJet Airlin...|   IAH| BTR|    False|          Texas|    Louisiana|
|2019-01-01|ExpressJet Airlin...|   ORD| SDF|    False|       Illinois|     Kentucky|
|2019-01-01|ExpressJet Airlin...|   LFT| IAH|    False|      Louisiana|        Texas|
|2019-01-01|ExpressJet Airlin...|   IAH| VPS|     True|          Texas|      Florida|
|2019-01-01|ExpressJet Airlin...|   GPT| IAH|    False|    Mississippi|        Texas|
+----------+--------------------+------+----+---------+---------------+-------------+
only showing top 5 rows



In [25]:
flight_df.repartition(1).write.csv("/Users/fengzheng/Desktop/output")